In [1]:
import pandas as pd

stock_table = pd.read_csv('../../data/explore/stock_url_list.csv')
stock_table.head()

,Stock,ราคาล่าสุด(Home),Sector,กรรมการบริษัท,งบการเงิน,บริษัท/หลักทรัพย์,ผู้ถือหุ้นรายใหญ่,ราคาย้อนหลัง
0,CHOTI,/C13_FastQuote_Main.jsp?txtSymbol=CHOTI,AGRI,/C13_FastQuote_Main.jsp?txtSymbol=CHOTI&ssoPag...,/C13_FastQuote_Main.jsp?txtSymbol=CHOTI&ssoPag...,/C13_FastQuote_Main.jsp?txtSymbol=CHOTI&ssoPag...,/C13_FastQuote_Main.jsp?txtSymbol=CHOTI&ssoPag...,/C13_FastQuote_Main.jsp?txtSymbol=CHOTI&ssoPag...
1,EE,/C13_FastQuote_Main.jsp?txtSymbol=EE,AGRI,/C13_FastQuote_Main.jsp?txtSymbol=EE&ssoPageId...,/C13_FastQuote_Main.jsp?txtSymbol=EE&ssoPageId...,/C13_FastQuote_Main.jsp?txtSymbol=EE&ssoPageId...,/C13_FastQuote_Main.jsp?txtSymbol=EE&ssoPageId...,/C13_FastQuote_Main.jsp?txtSymbol=EE&ssoPageId...
2,GFPT,/C13_FastQuote_Main.jsp?txtSymbol=GFPT,AGRI,/C13_FastQuote_Main.jsp?txtSymbol=GFPT&ssoPage...,/C13_FastQuote_Main.jsp?txtSymbol=GFPT&ssoPage...,/C13_FastQuote_Main.jsp?txtSymbol=GFPT&ssoPage...,/C13_FastQuote_Main.jsp?txtSymbol=GFPT&ssoPage...,/C13_FastQuote_Main.jsp?txtSymbol=GFPT&ssoPage...
3,LEE,/C13_FastQuote_Main.jsp?txtSymbol=LEE,AGRI,/C13_FastQuote_Main.jsp?txtSymbol=LEE&ssoPageI...,/C13_FastQuote_Main.jsp?txtSymbol=LEE&ssoPageI...,/C13_FastQuote_Main.jsp?txtSymbol=LEE&ssoPageI...,/C13_FastQuote_Main.jsp?txtSymbol=LEE&ssoPageI...,/C13_FastQuote_Main.jsp?txtSymbol=LEE&ssoPageI...
4,STA,/C13_FastQuote_Main.jsp?txtSymbol=STA,AGRI,/C13_FastQuote_Main.jsp?txtSymbol=STA&ssoPageI...,/C13_FastQuote_Main.jsp?txtSymbol=STA&ssoPageI...,/C13_FastQuote_Main.jsp?txtSymbol=STA&ssoPageI...,/C13_FastQuote_Main.jsp?txtSymbol=STA&ssoPageI...,/C13_FastQuote_Main.jsp?txtSymbol=STA&ssoPageI...


# งบการเงิน

ดูดข้อมูลหน้านี้จาก Settrade

<img width="400" src="../img/financial_report.png" >

ข้อมูลจากตาราง stock ทำให้เรารู้ url ของข้อมูลที่เราจะไปดึงมาแล้ว ดังนั้นทำฟังก์ชั่นสำหรับดึงข้อมูลงบการเงิน จะได้

In [2]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import re

In [3]:
def is_float(input):
    try:
        num = float(input)
    except ValueError:
        return False
    return True

In [4]:
def getTableHeadData(head):
    data_head = head.find_all('strong')
    date_html = data_head[1:len(data_head)]
    date_text = [ list(html.stripped_strings) for html in date_html ]
    dates = [ year_date[1] if len(year_date) > 0 else '-' for year_date in date_text ]
    return [data_head[0].text,dates]

In [5]:
def getTableSecondHeadData(head):
    data_head = head.find_all('strong')
    date_html = data_head[1:len(data_head)]
    date_text = [ list(html.stripped_strings) for html in date_html ]
    dates = [ year_date[0] if len(year_date) > 0 else '-' for year_date in date_text ]
    return [data_head[0].text,dates]

In [11]:
def getTableRowText(row):
    data_row = row.find_all('td')
    data_list = [ list(data.stripped_strings) for data in data_row if len(data) > 0 ]
    data_text = [ data[0] if len(data) > 0 else '-' for data in data_list ]
    return [data_text[0],data_text[1:len(data_text)]]

In [125]:
def getStockDetail(url,stock):
    page = requests.get(url) # ดูดหน้าเว็บมา
    
    if page.status_code == 200 :
        # All HTML of Page
        soup = BeautifulSoup(page.content, "html5lib")
        # ซอยข้อมูลทั้ง page เป็นทีละส่วนๆ แล้วลงลึกไปเอาข้อมูลทีละจุดๆ
        html = list(soup.children)[1]
        head = list(html.children)[0]
        body = list(html.children)[2]
        if len(body.find_all(class_='table-info')) > 0:
            # ***
            table_info = list(body.find(class_='table-info'))
            table_header1 = table_info[3]
            table_body1 = table_info[5]
            table_body_row1 = table_body1.find_all('tr')
            table_header2 = table_info[6]
            table_body2 = table_info[8]
            table_body_row2 = table_body2.find_all('tr')

            # -------------- Top Table ----------------
            dates = getTableHeadData(table_header1) # Date : งวดงบการเงิน ณ วันที่
            asset = getTableRowText(table_body_row1[1]) # Asset : สินทรัพย์รวม
            debt = getTableRowText(table_body_row1[2]) # Debt : หนี้สินรวม
            equity = getTableRowText(table_body_row1[3]) # Equity : ส่วนของผู้ถือหุ้น
            paid_up = getTableRowText(table_body_row1[4]) # Paid-up Capital : มูลค่าหุ้นที่เรียกชำระแล้ว
            revenue = getTableRowText(table_body_row1[5]) # Revenue : รายได้รวม
            net_profit = getTableRowText(table_body_row1[6]) # Net Profit : กำไรสุทธิ
            eps = getTableRowText(table_body_row1[7]) # EPS (Baht) : กำไรต่อหุ้น (บาท)
            roa = getTableRowText(table_body_row1[9]) # ROA(%)
            roe = getTableRowText(table_body_row1[10]) # ROE(%)
            net_profit_margin = getTableRowText(table_body_row1[11]) # Net Profit Margin(%) : อัตรากำไรสุทธิ(%)

            # -------------- Bottom Table ----------------
            dates2 = getTableSecondHeadData(table_header2) # Date : ค่าสถิติสำคัญ ณ วันที่
            last_price = getTableRowText(table_body_row2[0]) # Last Price(Baht) : ราคาล่าสุด(บาท)
            market_cap = getTableRowText(table_body_row2[1]) # Market Cap. : มูลค่าหลักทรัพย์ตามราคาตลาด
            f_s_period = getTableRowText(table_body_row2[2]) # F/S Period (As of date) : วันที่ของงบการเงินที่ใช้คำนวณค่าสถิติ
            p_e = getTableRowText(table_body_row2[3]) # P/E (เท่า)
            pbv = getTableRowText(table_body_row2[4]) # P/BV (เท่า)
            book_value_per_share = getTableRowText(table_body_row2[5]) # Book Value per share (Baht) : มูลค่าหุ้นทางบัญชีต่อหุ้น (บาท)
            dvd_yield = getTableRowText(table_body_row2[6]) # Dvd. Yield(%) : อัตราส่วนเงินปันผลตอบแทน(%)

            # Convert to DataFrame         
            report_df1 = pd.DataFrame({dates[0]: dates[1],  
                                       asset[0]: asset[1], 
                                       debt[0]: debt[1],
                                       equity[0]: equity[1],
                                       paid_up[0]: paid_up[1],
                                       revenue[0]: revenue[1],
                                       net_profit[0]: net_profit[1],
                                       eps[0]: eps[1],
                                       roa[0]: roa[1],
                                       roe[0]: roe[1],
                                       net_profit_margin[0]: net_profit_margin[1],
                                    })
            report_df2 = pd.DataFrame({dates2[0]: dates2[1],  
                                       last_price[0]:last_price[1], 
                                       market_cap[0]: market_cap[1],
                                       f_s_period[0]: f_s_period[1],
                                       p_e[0]: p_e[1],
                                       pbv[0]: pbv[1],
                                       book_value_per_share[0]: book_value_per_share[1],
                                       dvd_yield[0]: dvd_yield[1],
                                    })
            report_df1 =  report_df1[~report_df1.eq('-', axis='index')].dropna(axis=0, how='all')
            report_df2 =  report_df2[~report_df2.eq('-', axis='index')].dropna(axis=0, how='all')
            report_df1['Stock'],report_df2['Stock'] = stock,stock
            return [True,report_df1,report_df2]
        else:
            return [False]
    else:
        print("Can't get content from this URL!!!")
        return [False]

In [126]:
prefix_url = 'https://www.settrade.com'
suffix_url = '/C13_FastQuote_Main.jsp?txtSymbol=S+%26+J&ssoPageId=13&selectPage=6'
settrade_url = prefix_url + suffix_url

In [127]:
sj = getStockDetail(settrade_url,'S & J')
sj[0]

False

In [129]:
prefix_url = 'https://www.settrade.com'
suffix_url = '/C13_FastQuote_Main.jsp?txtSymbol=VPO&ssoPageId=13&selectPage=6'
settrade_url = prefix_url + suffix_url

In [130]:
vpo = getStockDetail(settrade_url,'VPO')
vpo[1]

,ROA(%),ROE(%),กำไรต่อหุ้น (บาท),กำไรสุทธิ,งวดงบการเงิน ณ วันที่,มูลค่าหุ้นที่เรียกชำระแล้ว,รายได้รวม,สินทรัพย์รวม,ส่วนของผู้ถือหุ้น,หนี้สินรวม,อัตรากำไรสุทธิ(%),Stock
0,8.21,8.54,0.11,93.83,31/12/2557,940.00,"2,304.89","1,994.52","1,279.22",715.29,4.07,VPO
1,-5.12,-10.48,-0.13,-124.99,31/12/2558,940.00,"1,481.80","1,729.55","1,107.23",622.32,-8.44,VPO
2,-10.77,-21.14,-0.23,-212.37,31/12/2559,940.00,"1,183.38","1,708.69",902.09,806.60,-17.95,VPO
3,-10.25,-24.31,-0.21,-195.89,31/12/2560,940.00,"1,391.19","1,485.90",709.42,776.48,-14.08,VPO


In [131]:
vpo[2]

,P/BV (เท่า),P/E (เท่า),ค่าสถิติสำคัญ ณ วันที่,มูลค่าหลักทรัพย์ตามราคาตลาด,มูลค่าหุ้นทางบัญชีต่อหุ้น (บาท),ราคาล่าสุด(บาท),วันที่ของงบการเงินที่ใช้คำนวณค่าสถิติ,อัตราส่วนเงินปันผลตอบแทน(%),Stock
0,N/A,21.74,30/12/2557,"2,744.80",N.A.,2.92,NaN,N/A,VPO
1,1.00,N/A,30/12/2558,"1,193.80",1.26,1.27,30/09/2558,3.94,VPO
2,1.46,N/A,30/12/2559,"1,381.80",1.01,1.47,30/09/2559,N/A,VPO
3,1.01,N/A,29/12/2560,742.60,0.78,0.79,30/09/2560,N/A,VPO
4,1.06,N/A,26/04/2561,752.00,0.75,0.80,31/12/2560,N/A,VPO


ดูของทุกหุ้นมาต่อกันรวมกันในตารางเดียว

In [132]:
prefix_url = 'https://www.settrade.com'
report1_frames = []
report2_frames = []

for index,stock in stock_table.iterrows():
    suffix_url = stock['งบการเงิน']
    settrade_url = prefix_url + suffix_url
    stock_df = getStockDetail(settrade_url,stock['Stock'])
    if stock_df[0] == False :
        status = 'Fail!'
    else:
        status = 'OK'
        report1_frames.append(stock_df[1])
        report2_frames.append(stock_df[2])
    print(stock['Stock'],status)

report1 = pd.concat(report1_frames)
report2 = pd.concat(report2_frames)

CHOTI OK
EE OK
GFPT OK
LEE OK
STA OK
TLUXE OK
TRUBB OK
TWPC OK
UPOIC OK
UVAN OK
VPO OK
APURE OK
ASIAN OK
BR OK
BRR OK
CBG OK
CFRESH OK
CM OK
CPF OK
CPI OK
F&D OK
HTC OK
ICHI OK
KBS OK
KSL OK
KTIS OK
LST OK
M OK
MALEE OK
MINT OK
OISHI OK
PB OK
PM OK
PRG OK
SAPPE OK
SAUCE OK
SFP OK
SNP OK
SORKON OK
SSC OK
SSF OK
SST OK
TC OK
TFG OK
TFMAMA OK
TIPCO OK
TKN OK
TU OK
TVO OK
AFC OK
BTNC OK
CPH OK
CPL OK
DIGI OK
ICC OK
LTX OK
NC OK
PAF OK
PDJ OK
PG OK
SABINA OK
SAWANG OK
SUC OK
TNL OK
TPCORP OK
TR OK
TTI OK
TTL OK
TTTM OK
UPF OK
UT OK
WACOAL OK
ACC OK
AJA OK
DTCI OK
FANCY OK
KYE OK
L&E OK
MODERN OK
OGC OK
ROCK OK
SIAM OK
TSR OK
DDD OK
JCT OK
OCC OK
S & J Fail!
STHAI OK
TNR OK
TOG OK
BAY OK
BBL OK
CIMBT OK
KBANK OK
KKP OK
KTB OK
LHFG OK
SCB OK
TCAP OK
TISCO OK
TMB OK
AEC OK
AEONTS OK
AMANAH OK
ASAP OK
ASK OK
ASP OK
BFIT OK
CGH OK
CNS OK
ECL OK
FNS OK
FSS OK
GBX OK
GL OK
IFS OK
JMT OK
KCAR OK
KGI OK
KTC OK
MBKET OK
MFC OK
ML OK
MTLS OK
PE OK
PL OK
S11 OK
SAWAD OK
THANI OK
TK OK
TNITY OK
UOBKH OK

In [135]:
report1[0:10]

,ROA(%),ROE(%),กำไรต่อหุ้น (บาท),กำไรสุทธิ,งวดงบการเงิน ณ วันที่,มูลค่าหุ้นที่เรียกชำระแล้ว,รายได้รวม,สินทรัพย์รวม,ส่วนของผู้ถือหุ้น,หนี้สินรวม,อัตรากำไรสุทธิ(%),Stock
0,-1.76,0.04,0.07,0.49,31/12/2557,75.00,"2,866.64","1,585.91","1,221.21",364.60,0.02,CHOTI
1,2.51,2.57,4.19,31.43,31/12/2558,75.00,"2,335.20","1,688.20","1,228.46",459.63,1.35,CHOTI
2,8.45,9.85,16.69,125.16,31/12/2559,75.00,"3,484.25","1,768.07","1,313.75",454.19,3.59,CHOTI
3,1.14,1.02,1.71,12.81,31/12/2560,75.00,"3,506.62","2,052.59","1,207.33",845.13,0.37,CHOTI
0,0.09,0.02,0.00,0.29,31/12/2557,"2,780.00",39.23,"1,557.73","1,553.75",3.98,0.75,EE
1,-6.81,-6.82,-0.04,-102.52,31/12/2558,"2,780.00",10.32,"1,455.12","1,451.23",3.89,-993.50,EE
2,-10.59,-10.62,-0.05,-146.37,31/12/2559,"2,780.00",3.87,"1,308.76","1,304.86",3.90,"-3,786.40",EE
3,3.43,3.46,0.02,46.08,31/12/2560,"2,780.00",102.39,"1,390.10","1,358.94",31.16,45.01,EE
0,14.17,22.24,1.42,"1,779.59",31/12/2557,"1,253.82","18,401.01","13,737.41","8,633.26","5,004.43",9.67,GFPT
1,9.17,13.31,0.95,"1,194.92",31/12/2558,"1,253.82","16,950.47","14,976.32","9,326.74","5,533.07",7.05,GFPT


In [137]:
report2[0:10]

,NAV,P/BV (เท่า),P/E (เท่า),P/NAV (เท่า),Stock,ค่าสถิติสำคัญ ณ วันที่,มูลค่าหลักทรัพย์ตามราคาตลาด,มูลค่าหุ้นทางบัญชีต่อหุ้น (บาท),ราคาล่าสุด(บาท),วันที่ของงบการเงินที่ใช้คำนวณค่าสถิติ,อัตราส่วนเงินปันผลตอบแทน(%)
0,NaN,1.05,41.73,NaN,CHOTI,30/12/2557,"1,271.25",162.11,169.50,30/09/2557,1.77
1,NaN,0.76,N/A,NaN,CHOTI,30/12/2558,888.75,156.67,118.50,30/09/2558,N/A
2,NaN,1.12,11.50,NaN,CHOTI,30/12/2559,"1,421.25",169.57,189.50,30/09/2559,2.11
3,NaN,1.01,43.76,NaN,CHOTI,29/12/2560,"1,181.25",156.20,157.50,30/09/2560,10.16
4,NaN,0.88,83.13,NaN,CHOTI,26/04/2561,"1,065.00",160.98,142.00,31/12/2560,0.70
0,NaN,2.47,211.50,NaN,EE,30/12/2557,"3,864.20",0.56,1.39,30/09/2557,N/A
1,NaN,1.88,N/A,NaN,EE,30/12/2558,"2,807.80",0.54,1.01,30/09/2558,N/A
2,NaN,2.25,N/A,NaN,EE,30/12/2559,"3,058.00",0.49,1.10,30/09/2559,N/A
3,NaN,1.83,N/A,NaN,EE,29/12/2560,"2,446.40",0.48,0.88,30/09/2560,N/A
4,NaN,1.35,39.81,NaN,EE,26/04/2561,"1,834.80",0.49,0.66,31/12/2560,N/A


In [138]:
report1.to_csv('../../data/explore/fin_statement_balance.csv', index=False)

In [139]:
report2.to_csv('../../data/explore/fin_statement_statistic.csv', index=False)

### งบการเงิน
- หนี้สินรวม (2017,2016,...) - Home
- เจ้าหนี้การค้าและเจ้าหนี้อื่น (2017,2016,...) - งบเต็ม
- ส่วนของผู้ถือหุ้น (2017,2016,...) - Home
- หนี่สินที่มีภาระดอกเบี้ย (2017,2016,...) - งบเต็ม ซึ่งประกอบไปด้วย
    - เงินเบิกเกินบัญชีและเงินกู้ยืมระยะสั้นจากสถาบันการเงิน(งบเต็ม)
    - ส่วนของหนี้สินระยะยาวที่ถึงกำหนดชำระภายในหนึ่งปี(งบเต็ม) : อาจจะสนแค่ เงินกู้ยืมระยะยาว-สถาบันการเงิน+ตราสารหนี้
    - หนี้สินระยะยาว-สุทธิจากส่วนที่ถึงกำหนดชำระภายในหนึ่งปี(งบเต็ม) : อาจจะสนแค่ เงินกู้ยืมระยะยาว-สถาบันการเงิน+ตราสารหนี้
    
### ตรวจสอบส่วนของผู้ถือหุ้นว่ามาจากกำไรหรือระดมทุนเพิ่ม
- มูลค่าหุ้นที่เรียกชำระแล้ว (2017,2016,2015,...) - Home

In [141]:
report2['NAV'].unique()

array([nan, '10.00', '10.23', '10.27', '10.13', '10.11', 'N.A.', '12.26',
       '12.12', '12.29', '12.49', '11.48', '11.75', '11.66', '11.05',
       '11.08', '13.31', '10.34', '10.30', '10.26', '10.35', '10.80',
       '10.92', '11.03', '10.79', '10.74', '9.97', '8.90', '7.68', '7.29',
       '6.46', '8.96', '10.71', '10.46', '9.22', '9.23', '11.55', '11.80',
       '12.79', '12.92', '12.86', '10.14', '8.74', '8.81', '11.00', '9.84',
       '9.93', '9.87', '9.66', '10.09', '10.15', '10.19', '10.21', '9.81',
       '9.88', '8.69', '8.51', '10.60', '10.84', '10.90', '10.93', '10.16',
       '10.22', '10.50', '10.48', '10.54', '10.59', '12.32', '12.34',
       '10.49', '10.56', '10.72', '11.27', '11.56', '11.59', '10.97',
       '11.12', '11.38', '11.44', '10.29', '10.45', '9.40', '9.38',
       '10.18', '10.20', '10.52', '11.39', '11.49', '12.91', '13.07',
       '13.26', '13.30', '13.33', '8.28', '8.48', '8.00', '12.07', '12.24',
       '12.21', '12.03', '5.29', '5.30', '5.23', '5.18'